EXERCISE 1 - MUST OPEN SAME SCRIPT IN SEPERATE WINDOW 

In [ ]:
import socket 
import threading 

client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) 
server_address = ('localhost', 65433) 
client_socket.bind(('', 0))

username = input("Name: ")

def recieveMessages():
    while True:
        try:
            data, _ = client_socket.recvfrom(2048)
            message = data.decode().strip()
            print(f"{message}\n", end="")
        except:
            break

threading.Thread(target=recieveMessages, daemon=True).start()
client_socket.sendto(f"{username} has joined.".encode(), ('localhost', 65433))

while True:
    message = input("> ")
    if message.lower() == "exit":
        break
    full_message = f"{username}: {message}"
    client_socket.sendto(full_message.encode(), ('localhost', 65433))

client_socket.close()

![title](Images/Results_2.png)

![title](Images/Results_3.png)

EXCERCISE 2,3,4 - CLIENT SIDE

In [7]:
import socket


def caesarCipher(text, shift):
    result = ""
    for i in range(len(text)):
        char = text[i]
        if char.isupper():
            result += chr((ord(char) + shift - 65) % 26 + 65)
        elif char.islower():
            result += chr((ord(char) + shift - 97) % 26 + 97)
        else:
            result +=char

    return result

def decrypt(text, shift):
    return caesarCipher(text, -shift)

shift = 4

client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_address = ('localhost', 65533)

username = input("username: ").strip()
password = input("password: ").strip()
credentials = f"{username} {password}"
print(f"Sending credentials '{credentials}'")

client_socket.sendto(caesarCipher(credentials, shift).encode(), server_address)

data, client_address =  client_socket.recvfrom(2048)
response = decrypt(data.decode(), shift)
print(f"Server: {response}")

if "failed" in response.lower():
    print("authentication failed. leaving...")
    client_socket.close()
    exit()

print("type exit to leave")


while True:
    message = input("you: ") 

    if message.lower() == "exit":

        client_socket.sendto(caesarCipher("client left",shift).encode(), server_address)
        data, client_address = client_socket.recvfrom(2048)
        print(f"Server says: {decrypt(data.decode(), shift)}")
        client_socket.close()

        break

    client_socket.sendto(caesarCipher(message, shift).encode(), server_address)

    data, client_address =  client_socket.recvfrom(2048)
    print(f"Server says: {decrypt(data.decode(),shift)}")

client_socket.close()



Sending credentials 'username1 password1'
Server: authentication granted
type exit to leave
Server says: heyyy
Server says: No problem ;)
Server says: byeeeeeeee!


![title](Images/Results_5.png)

EXERCISE 5 - client side

In [8]:
import socket
import requests

api_url_university = "https://api.open-meteo.com/v1/forecast?latitude=51.4746&longitude=-0.0355&current_weather=true"
api_url_library = "https://api.open-meteo.com/v1/forecast?latitude=51.5299&longitude=-0.1275&current_weather=true"

def fetch_temperature(api_url):
    response = requests.get(api_url)
    if response.status_code == 200:
        weather_data = response.json()
        return weather_data["current_weather"]["temperature"]
    else:
        return None 

temperature_university = fetch_temperature(api_url_university)
temperature_library = fetch_temperature(api_url_library)

if temperature_university is not None and temperature_library is not None:
    if temperature_university > temperature_library:
        comparison = "The goldsmiths is warmer than the british library."
    elif temperature_university < temperature_library:
        comparison = "The british library is warmer than the university."
    else:
        comparison = "locations have the same temperature."

    message = f"Weather Update:\n goldsmiths: {temperature_university}°C\n british library: {temperature_library}°C\n{comparison}"
else:
    message = "failed to get weather"

client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_address = ('localhost', 65534) 

client_socket.sendto(message.encode(), server_address)  

client_socket.close()


EXERCISE 6 - CLIENT SIDE

In [9]:
import socket
import requests

api_url_joke = "https://official-joke-api.appspot.com/random_joke"

def fetch_joke(api_url):
    response = requests.get(api_url)
    if response.status_code == 200:
        joke_data = response.json()
        return f"{joke_data['setup']} - {joke_data['punchline']}"
    else:
        return "failed to get the joke lol "

joke = fetch_joke(api_url_joke)

client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_address = ('localhost', 65535) 

client_socket.sendto(joke.encode(), server_address)

client_socket.close()
